In [1]:
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [2]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

--2024-09-09 22:57:52--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.41K  --.-KB/s    in 0s      

2024-09-09 22:57:52 (27.9 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3495/3495]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/c9a4704/releases/h5py-install.sh
+ [[ https://githu

In [3]:
import numpy as np
from ufl import sym, grad, Identity, tr, inner, Measure, TestFunction, TrialFunction

from mpi4py import MPI

from dolfinx import fem, io
import dolfinx.fem.petsc
from dolfinx.io import gmshio
# MPI communicator
comm = MPI.COMM_WORLD

file_name = "cylinder.msh"
mesh, cell_tags, _ = gmshio.read_from_msh(file_name, comm,
                                          rank=0, gdim=3)



dim = mesh.topology.dim

degree = 1
shape = (dim,)  # this means we want a vector field of size `dim`
V = fem.functionspace(mesh, ("P", degree, shape))

u_sol = fem.Function(V, name="Displacement")

# Material parameters

W = dolfinx.fem.functionspace(mesh, ("DG", 0))
E = dolfinx.fem.Function(W)
E.x.array[:] = 25e3 #mortero
E.x.array[cell_tags.find(1)] = 70e3 #agregado

nu = fem.Constant(mesh, 0.3)

lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2 / (1 + nu)


def epsilon(v):
    return sym(grad(v))


def sigma(v):
    return lmbda * tr(epsilon(v)) * Identity(dim) + 2 * mu * epsilon(v)


u = TrialFunction(V)
v = TestFunction(V)

rho = 2e-3
g = 9.81
f = fem.Constant(mesh, np.array([0,0, -rho * g]))

dx = Measure("dx", domain=mesh)
a = inner(sigma(u), epsilon(v)) * dx
L = inner(f, v) * dx


def arriba(x):
    return np.isclose(x[2], 200)


def abajo(x):
    return np.isclose(x[2], 0.0)


arriba_dofs = fem.locate_dofs_geometrical(V, arriba)
abajo_dofs = fem.locate_dofs_geometrical(V, abajo)
deltaU = np.zeros((3,))
deltaU[2] = -0.1

bcs = [
    fem.dirichletbc(deltaU, arriba_dofs, V),
    fem.dirichletbc(np.zeros((3,)), abajo_dofs, V),
]

problem = fem.petsc.LinearProblem(
    a, L, u=u_sol, bcs=bcs, petsc_options={"ksp_type": "cg", "pc_type": "hypre","ksp_rtol ": 1e-10 , "ksp_atol ": 1e-15}
)
problem.solve()


vtk = io.VTKFile(mesh.comm, "linear_elasticity.pvd", "w")
vtk.write_function(u_sol)
vtk.close()
